In [1]:
import os
import json
import requests
import datetime

sat_api_url = "https://landsatlook.usgs.gov/sat-api"

In [2]:
def query_satapi(query):
    headers = {
            "Content-Type": "application/json",
            "Accept-Encoding": "gzip",
            "Accept": "application/geo+json",
        }

    url = f"{sat_api_url}/stac/search"
    data = requests.post(url, headers=headers, json=query).json()
    
    return data

In [4]:
min_cloud = 0
max_cloud = 50

date_min="2015-03-01"
date_max="2015-09-30"

# format datatime for query
start_date = datetime.datetime.strptime(date_min, "%Y-%m-%d")
end_date = datetime.datetime.strptime(date_max, "%Y-%m-%d") 
start = start_date.strftime("%Y-%m-%dT00:00:00Z")
end = end_date.strftime("%Y-%m-%dT23:59:59Z")


In [13]:
# Get a small sample first
sample_query = {
    "time": f"{start}/{end}",
    "bbox": [-101,40,-100,80],
    "limit": 10
}
sample = query_satapi(sample_query)


In [5]:
# Take a look at the sample
# drill down to see what properties are available
sample.keys()
sample['features'][0].keys()
print(sample['features'][0]['properties'].keys())
# eo:sun_azimuth and eo:sun_elevation
sample['features'][0]['properties']['eo:sun_azimuth']


dict_keys(['collection', 'eo:gsd', 'eo:platform', 'eo:off_nadir', 'datetime', 'eo:cloud_cover', 'eo:sun_azimuth', 'eo:sun_elevation', 'eo:instrument', 'landsat:cloud_cover_land', 'landsat:wrs_type', 'landsat:wrs_path', 'landsat:wrs_row', 'landsat:scene_id', 'landsat:collection_category', 'landsat:collection_number', 'eo:bands'])


166.84936113

In [9]:
data['links']

[{'rel': 'next',
  'title': 'Next page of results',
  'href': 'https://landsatlook.usgs.gov/sat-api/stac/search?datetime=%222015-03-01T00%3A00%3A00Z%2F2015-09-30T23%3A59%3A59Z%22&query=%7B%22bbox%22%3A%5B-180%2C40%2C180%2C80%5D%2C%22eo%3Acloud_cover%22%3A%7B%22gte%22%3A0%2C%22lt%22%3A50%7D%2C%22collection%22%3A%7B%22eq%22%3A%22landsat-c2l2-sr%22%7D%7D&page=2&limit=500'}]

In [14]:
# Testing collection filter

full_query = {
    "time": f"{start}/{end}",
    "bbox": [-180, 40, 180, 80],
    "query": { 
        "collection":{"eq": "landsat-c2l1"},
    },
    "limit": 5 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
}

data = query_satapi(full_query)

In [13]:
data['meta']['found']

0

In [43]:
# set the min and max sun azimuth if you want to filter
min_az = 100
max_az = 160

full_query = {
    "time": f"{start}/{end}",
    "bbox": [-180, 40, 180, 80],
    "query": {
        #"eo:sun_azimuth": {"gte": min_az, "lt": max_az}, #Remove comment to enable
        "eo:platform": {"eq": "LANDSAT_8"},
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud},
        "collection":{"eq": "landsat-c2l2-sr"},
        "landsat:collection_category":{"eq": "T1"}
    },
    "limit": 5 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
}

# this is the maximum possible
data = query_satapi(full_query)


In [49]:
scenes = data['meta']['found']
print(f'{scenes} scenes')

26741 scenes


In [50]:
# for all  scences * years * bands, estimate
scenes_all = scenes * 5 * 5
print(f'{scenes_all} scenes')

668525 scenes


In [51]:
# estimate of 70 MB per band, convert to TB
# scenes * 70 MB
size = scenes_all * (70/1024)
print(f'{size:,} GB')

45,699.951171875 GB


In [52]:
# Assuming requesting each file entirely once
cost_request = round((scenes_all / 1000) * 0.0004, 2)
cost_bandwidth = round(size * 0.02, 2)
print(f'${cost_request} ${cost_bandwidth:,}')

$0.27 $914.0


In [15]:
def estimate_costs(scenes, bands = 5, years = 5):
    data = {}
    data['scene'] = scenes
    data['bands'] = bands
    data['years'] = years
    data['scenes_tot'] = scenes * bands * years
    #print(f'{data["scenes_tot"]} bands X scenes X years')
    data['size'] = data['scenes_tot'] * (90/1024) #90 MB per band per scene estimate is a more conservative estimate
    #print(f'Size {data["size"]:,} GB')
    data['cost_request'] = round((data['scenes_tot'] / 1000) * 0.0004, 2)
    data['cost_bandwidth'] = round(data['size'] * 0.02, 2)
    #print(f'Cost ${data["cost_request"]} ${data["cost_bandwidth"]:,}')
    
    return data

## Actual Queries

This section runs a query for each year getting the actual matches. While we return only the scene counts, the same query results if parsed and paged through could return the actual list of scenes for retrieval.

In [36]:
def query_year(year):
    '''Given the year, finds the number of scenes matching the query and returns it.'''
    date_min = '-'.join([str(year), "03-01"])
    date_max = '-'.join([str(year), "09-30"])
    start_date = datetime.datetime.strptime(date_min, "%Y-%m-%d")
    end_date = datetime.datetime.strptime(date_max, "%Y-%m-%d") 
    start = start_date.strftime("%Y-%m-%dT00:00:00Z")
    end = end_date.strftime("%Y-%m-%dT23:59:59Z")
    
    query = {
    "time": f"{start}/{end}",
    "query": {
        "bbox":[-180,40,180,80],
        "eo:cloud_cover": {"gte": min_cloud, "lt": max_cloud},
        "collection":{"eq": "landsat-c2l2-sr"}
        },
    "limit": 500 # We limit to 500 items per Page (requests) to make sure sat-api doesn't fail to return big features collection
    }
    
    data = query_satapi(query)
    scenes = data['meta']['found']
    
    return scenes

In [39]:
scene_totals = [query_year(year) for year in range(2015,2020)]

In [43]:
# compare actual to estimated scenes
(sum(scene_totals) * 5) - scenes_all

33105

In [ ]:
# TODO:
# Limit to over land only

In [21]:
import pandas as pd
cost_estimate = pd.DataFrame.from_dict([estimate_costs(scenes, 5, 1) for scenes in [17,17,15]])
print(cost_estimate.head())
cost_estimate.sum()


   scene  bands  years  scenes_tot      size  cost_request  cost_bandwidth
0     17      5      1          85  7.470703           0.0            0.15
1     17      5      1          85  7.470703           0.0            0.15
2     15      5      1          75  6.591797           0.0            0.13


scene              49.000000
bands              15.000000
years               3.000000
scenes_tot        245.000000
size               21.533203
cost_request        0.000000
cost_bandwidth      0.430000
dtype: float64